In [2]:
import os
import json
import psycopg2
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import folium
import folium.plugins
import fiona
from pyproj import Proj, transform
import datetime

In [3]:
import matplotlib.cm as cmx
import matplotlib.colors as colors


def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return r, g, b


def style_function(feature, n_colors):
    cid = feature['properties']['cid']
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#red' if cid is None else "rgb{}".format(rgb(0, n_colors, cid))
    }

def init_style_function(n_colors):
    return lambda feature: style_function(feature, n_colors)

In [4]:
conn = psycopg2.connect(dbname="gis", user="postgres", password="")
rapperswil_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1683921"
zurich_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1682248"
switzerland_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -51701"
hombrechtikon_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1682143"
staefa_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1682214"

rapperswil_location = [47.226, 8.818]
zurich_location = [47.3763, 8.5403]
hombrechtikon_location = [47.25212, 8.77038]
staefa_location = [47.24062, 8.72303]

epsg = 3857

polygon_query = rapperswil_polygon_query
location = rapperswil_location

In [5]:
shop_tags = ['mall', 'bakery', 'beverages', 'butcher', 'chocolate', 'coffee',
'confectionery', 'deli', 'frozen_food', 'greengrocer', 'healthfood',
'ice_cream', 'pasta', 'pastry', 'seafood', 'spices', 'tea', 'department_store',
'supermarket', 'bag', 'boutique', 'clothes', 'fashion', 'jewelry', 'leather',
'shoes', 'tailor', 'watches', 'chemist', 'cosmetics', 'hairdresser',
'medical_supply', 'electrical', 'hardware', 'electronics', 'sports',
'swimming_pool', 'collector', 'games', 'music', 'books', 'gift', 'stationery',
'ticket', 'laundry', 'pet', 'tobacco', 'toys']

amenity_tags = ['pub', 'bar', 'cafe', 'restaurant', 'pharmacy', 'bank', 'fast_food',
'food_court', 'ice_cream', 'library', 'ferry_terminal', 'clinic', 'doctors', 'hospital',
'pharmacy', 'veterinary', 'dentist', 'arts_centre', 'cinema',
'community_centre', 'casino', 'fountain', 'nightclub', 'studio', 'theatre',
'dojo', 'internet_cafe', 'marketplace', 'post_opffice', 'townhall']

leisure_tags = ['adult_gaming_centre', 'amusement_arcade', 'beach_resort',
'fitness_centre', 'garden', 'ice_rink', 'sports_centre', 'water_park']

landuse_tags = ['retail']

tags = {
    'shop_tags': shop_tags,
    'amenity_tags': amenity_tags,
    'leisure_tags': leisure_tags,
    'landuse_tags': landuse_tags
}

## Get all polygons with above tags

In [6]:
query = """
(SELECT way AS geometry FROM planet_osm_polygon
    WHERE (amenity = ANY(ARRAY{amenity_tags})
            OR shop = ANY(ARRAY{shop_tags})
            OR leisure = ANY(ARRAY{leisure_tags})
            OR landuse = ANY(ARRAY{landuse_tags}))
            
           AND access IS DISTINCT FROM 'private'
           AND st_within(way, ({polygon})))
           
UNION ALL

(SELECT polygon.way AS geometry FROM planet_osm_polygon AS polygon
    INNER JOIN planet_osm_point AS point
        ON st_within(point.way, polygon.way)
    WHERE (point.amenity = ANY(ARRAY{amenity_tags})
            OR point.shop = ANY(ARRAY{shop_tags})
            OR point.leisure = ANY(ARRAY{leisure_tags})
            OR point.landuse = ANY(ARRAY{landuse_tags}))
            
        AND point.access IS DISTINCT FROM 'private'
        AND st_within(point.way, ({polygon}))
        AND polygon.building IS NOT NULL)
        
""".format(polygon=polygon_query, **tags) 

poi_polygons = gpd.read_postgis(query, conn, geom_col='geometry')

poi_polygons.crs = fiona.crs.from_epsg(epsg)

m = folium.Map(location=location, zoom_start=16, tiles="cartodbpositron")

folium.plugins.Fullscreen().add_to(m)
folium.GeoJson(poi_polygons).add_to(m)

m

## Cluster above polygons with DBSCAN

In [7]:
cluster_query = """
SELECT polygon.geometry AS geometry,
       ST_ClusterDBSCAN(polygon.geometry, eps := 50, minpoints := 3) over () AS cid
FROM ({}) AS polygon
""".format(query)

poi_polygons_clustered = gpd.read_postgis(cluster_query, conn, geom_col='geometry')
n_clusters = len(poi_polygons_clustered.groupby('cid').cid.nunique())

poi_polygons_clustered.crs = fiona.crs.from_epsg(epsg)

m = folium.Map(location=location, zoom_start=16, tiles="cartodbpositron")

folium.plugins.Fullscreen().add_to(m)
folium.GeoJson(poi_polygons_clustered, style_function=init_style_function(n_clusters)).add_to(m)

m

## Create convex / concave hulls around clustered polygons

In [14]:
query_hulls = """
WITH clusters AS ({})
SELECT cid, ST_ConvexHull(ST_Union(geometry)) AS hull
FROM clusters
WHERE cid IS NOT NULL
GROUP BY cid
ORDER BY ST_Area(ST_ConvexHull(ST_Union(geometry))) DESC
LIMIT 1
""".format(cluster_query)

poi_polygons_clustered = gpd.read_postgis(query_hulls, conn, geom_col='hull')
n_clusters = len(poi_polygons_clustered.groupby('cid').cid.nunique())

poi_polygons_clustered.crs = fiona.crs.from_epsg(epsg)

m = folium.Map(location=location, zoom_start=16, tiles="cartodbpositron")

folium.plugins.Fullscreen().add_to(m)
folium.GeoJson(poi_polygons_clustered, style_function=init_style_function(n_clusters)).add_to(m)

m

In [8]:
#poi_polygons_clustered.to_crs(epsg=4326).to_file('aoi_rapperswil.geojson', driver='GeoJSON')

## Select all polygons inside concave hulls

In [9]:
polygons_inside_hulls_query = """
WITH hulls AS ({}) 
SELECT way as geometry FROM planet_osm_polygon AS polygon
    INNER JOIN hulls AS hulls ON ST_INTERSECTS(polygon.way, hulls.hull)
    WHERE polygon.building IS NOT NULL
""".format(query_hulls)

polygons_inside_hulls = gpd.read_postgis(polygons_inside_hulls_query, conn, geom_col='geometry')

polygons_inside_hulls.crs = fiona.crs.from_epsg(epsg)

m = folium.Map(location=location, zoom_start=16, tiles="cartodbpositron")

folium.GeoJson(polygons_inside_hulls).add_to(m)

m

## What happens if we UNION all polygons selected inside hulls?

In [10]:
polygons_inside_hulls_query = """
WITH hulls AS ({}) 
SELECT ST_UNION(way) as geometry FROM planet_osm_polygon AS polygon
    INNER JOIN hulls ON ST_INTERSECTS(polygon.way, hulls.hull)
    WHERE polygon.building IS NOT NULL
""".format(query_hulls)

polygons_inside_hulls = gpd.read_postgis(polygons_inside_hulls_query, conn, geom_col='geometry')

polygons_inside_hulls.crs = fiona.crs.from_epsg(epsg)

m = folium.Map(location=location, zoom_start=16, tiles="cartodbpositron")

folium.GeoJson(polygons_inside_hulls).add_to(m)

m

## Out of curiousity: Select all lines in side hulls

In [11]:
roads_query = """
WITH hulls AS ({}) 
SELECT way AS geometry FROM planet_osm_line AS ways
INNER JOIN hulls AS hulls ON ST_INTERSECTS(ways.way, hulls.hull) 
""".format(query_hulls)

roads = gpd.read_postgis(roads_query, conn, geom_col='geometry')

roads.crs = fiona.crs.from_epsg(epsg)

m = folium.Map(location=location, zoom_start=16, tiles="cartodbpositron")

folium.GeoJson(roads).add_to(m)

m

## Again create a concave hull around all polygons (which intersected with the first hull)

In [12]:
polygons_inside_hulls_query = """
WITH hulls AS ({}) 
SELECT cid, ST_ConcaveHull(ST_UNION(way), 0.80) as geometry FROM planet_osm_polygon AS polygon
    INNER JOIN hulls ON ST_INTERSECTS(polygon.way, hulls.hull)
    WHERE polygon.building = 'yes'
    GROUP BY cid;
""".format(query_hulls)

polygons_inside_hulls = gpd.read_postgis(polygons_inside_hulls_query, conn, geom_col='geometry')

polygons_inside_hulls.crs = fiona.crs.from_epsg(epsg)

m = folium.Map(location=location, zoom_start=16, tiles="cartodbpositron")

folium.GeoJson(polygons_inside_hulls).add_to(m)

m

In [13]:
#polygons_inside_hulls.to_crs(epsg=4326).to_file('aoi_zurich_2.geojson', driver='GeoJSON')